## Homework 9
Ron Cordell, Ted Dunmire, Filip Krunic 

-----

### Question 9.1 

*Write a basic MRJob implementation of the iterative PageRank algorithm
that takes sparse adjacency lists as input (as explored in HW 7).*

*Make sure that you implementation utilizes teleportation (1-damping/the number of nodes in the network), and further, distributes the mass of dangling nodes with each iteration
so that the output of each iteration is correctly normalized (sums to 1).*

#### Solution

In spirit of the previous homeworks, we implement a `driver` and subroutine modules to do various parts of the calculation. In particular, the driver will manage the overall process and the PageRank script will calculate the PageRank values for the dataset to be used. 

##### Driver 

Below is the driver for the PageRank process. 

In [37]:
%%writefile driver.py
from __future__ import division

from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.emr import EMRJobRunner

from init_pr import pageRank
from number_of_nodes import numNodes

import cPickle as pickle
from collections import defaultdict
from operator import itemgetter
import argparse 

# Storage files 
s3Bucket = 's3://ucb-mids-mls-networks/wikipedia/all-pages-indexed-out.txt'
dataFile = 'PageRank-test.txt'
emrID = 'j-8176IR8VX5LF'


def getName(obj, namespace):
	return [name for name in namespace if namespace[name] is obj]


def extractValues(job, runner):
	output = defaultdict(int)
	for line in runner.stream_output(): 
		key, value = job.parse_output_line(line)
		output[key] = value

	return output 


def dumpToFile(variable, filename):
	with open(filename, 'w') as f: 
		pickle.dump(variable, f)


def runJob(method, args, emr=False):
	job = method(args=args)

	methodName = getName(method, globals())[0]
	print '\n\t' + 'Running ' + methodName + '...'

	with job.make_runner() as runner: 

		runner.run()
		result = extractValues(job, runner)

		print '\t' + 'Complete: ' + methodName	

		return result 


if __name__ == '__main__':

	# Arguments 
	parser = argparse.ArgumentParser(description='Driver for PageRank in MRJob.')

	parser.add_argument('--emr', default=None, action='store_true',
					   help='Flag for using the EMR (and S3 bucket).')

	parser.add_argument('--iterations', default=10, 
							help='Number of iterations to use for PageRank.')

	args = parser.parse_args()

	# Pass 
	if args.emr: 
		# argInput = [s3Bucket, '-r', 'emr', '--no-strict-protocols', \
		# 		'--emr-job-flow-id', emrID]

		argInput = [s3Bucket, '-r', 'emr']

	else: 
		argInput = [dataFile]


	# Get node counts 
	totalNodeTuple = runJob(numNodes, args=argInput)
	totalNodes = totalNodeTuple.values()[0]
	
	print 'NUMNODES: %s' % (totalNodes)

	# Execute 
	topNodes = runJob(pageRank, args=argInput + ['--numberOfNodes=%s' % (totalNodes)] + \
					['--iterations=%s' % (args.iterations)])

	nodeTuples = [(key, round(value, 5)) for (key, value) in topNodes.iteritems()]
	sortedNodes = sorted(nodeTuples, key=itemgetter(1), reverse=True)

	# Emit 
	for k, v in sortedNodes:
		print 'ID: %s \t PR: %s' % (k, v)

Overwriting driver.py


##### Number of Nodes 

Below is the class to compute the number of nodes. This is important for PageRank as it could affect the values at the end if the number of nodes is not instantiated accurately. 

In [28]:
%%writefile number_of_nodes.py
from __future__ import division
 
from mrjob.job import MRJob
from mrjob.step import MRStep
 
from collections import defaultdict
from operator import itemgetter

class numNodes(MRJob):

	""" Counts the number of nodes. """

	def mapper(self, _, line):

		""" Emit raw nodes. """

		# Parse 
		line = line.split('\t')
		node = line[0]
		adjacencyList = eval(line[1])

		# Track 
		for neighbor in adjacencyList.keys(): 

		   # Emit raw nodes
		   yield neighbor, None


		# Pass values
		yield node, None


	def reducer_duplicates(self, node, _):

		""" Emit count for each unique node. """

		yield None, 1 


	def reducer_aggregate(self, _, totalNodes):

		""" Aggregate counts. """

		yield None, sum(totalNodes)


	def steps(self):

		return [MRStep(mapper=self.mapper, 
						reducer=self.reducer_duplicates), 

				MRStep(reducer=self.reducer_aggregate)]


if __name__ == '__main__':
	numNodes().run()				

Overwriting number_of_nodes.py


##### PageRank 

Below is the implementation for PageRank. It defaults to a set number of nodes based on either an estimate for EMR based jobs, or a `11` for the test dataset in question. 

In [14]:
%%writefile init_pr.py
from __future__ import division
 
from mrjob.job import MRJob
from mrjob.step import MRStep
 
from collections import defaultdict
from operator import itemgetter

class pageRank(MRJob):
 
     """ This class implements the page-rank calculation. """


     def configure_options(self):

          """ Load options for the class. """

          super(pageRank, self).configure_options()

          self.add_passthrough_option('--alpha',
               default=0.85, type=float, help='alpha: Dampening factor for teleportation in PageRank')

          self.add_passthrough_option('--iterations',
               default=10, type=int, help='iterations: number of iterations for PageRank')

          self.add_passthrough_option('--manualPower', 
               default=7, type=int, help='manualPower: order of magnitude for number of nodes.')

          self.add_passthrough_option('--numberOfNodes', 
               default=None, type=int, help='numberOfNodes: The number of nodes in your graph. Used for teleporation.')


     def load_options(self, args):

          """ Initializes the arguments for each class. """

          super(pageRank, self).load_options(args)

          self.alpha = self.options.alpha
          self.iterations = self.options.iterations

          # Check number of nodes 
          if self.options.numberOfNodes:
               self.numberOfNodes = self.options.numberOfNodes

          else:
               self.numberOfNodes = pow(10, self.options.manualPower)


     def mapper_init_pr(self, _, line):

          """ This initializes the PageRank algorithm by assembling the node list 
          for the initial PageRank values. """

          # Parse 
          line = line.split('\t')
          node = line[0]
          adjacencyList = eval(line[1])

          # Track 
          for neighbor in adjacencyList.keys(): 

               # Emit raw nodes
               yield neighbor, None


          # Pass values
          yield node, adjacencyList


     def reducer_init_pr(self, node, initTuple):

          """ This attaches initial PageRanks for the algorithm. """

          adjacencyList = dict()

          # Re-discover 
          for element in initTuple:
               if isinstance(element, dict):
                    adjacencyList = element 

          # Initialize PR
          PageRank = float(1) / float(self.numberOfNodes)

          # Emit
          yield node, (adjacencyList, PageRank)


     def mapper_iterate_pr(self, node, nodeTuple):

          """ This projects all of the PageRank weights for each node's neighbor. """

          adjacencyList, PageRank = nodeTuple

          if not adjacencyList:
               pass

          else: 

               # Emit PR 
               for neighbor in adjacencyList.keys(): 
                    yield neighbor, PageRank / len(adjacencyList)

          # Emit structure 
          yield node, adjacencyList


     def reducer_iterate_pr(self, node, PRNodeObject):

          """ This reconstructs the graph structure form the updated PageRanks. """

          updatedPR = 0

          # Combine PR 
          for value in PRNodeObject:
               if isinstance(value, dict):
                    adjacencyList = value 

               else: 
                    updatedPR += value 

          # Damping factor 
          updatedPR = ((1 - self.alpha) / self.numberOfNodes) + self.alpha * updatedPR

          # Emit 
          yield node, (adjacencyList, updatedPR)


     def mapper_sort(self, node, nodeTuple):

          """ Emits the page rank for each node. """

          adjacencyList, PageRank = nodeTuple

          yield None, (node, PageRank)


     def reducer_sort(self, _, PageRankPair):

          """ Keeps the top 100 PageRank values. """

          sortedList = []

          # Iterate and remove 
          for node, score in PageRankPair:

               sortedList.append((node, score))
               sortedList = sorted(sortedList, key=itemgetter(1), reverse=True)

               if len(sortedList) > 100: 
                    sortedList.pop()

          # Emit 
          for node, score in sortedList: 
               yield node, score


     def steps(self):

          """ Determines the steps for the job. Has two phases- initiate PR and iterate. """

          initializeStep = [

               MRStep(mapper=self.mapper_init_pr, 
                         reducer=self.reducer_init_pr)

          ]

          iterateStep = [

               MRStep(mapper=self.mapper_iterate_pr, 
                         reducer=self.reducer_iterate_pr)         

          ]

          sortStep = [

               MRStep(mapper=self.mapper_sort, 
                         reducer=self.reducer_sort)

          ]

          return initializeStep + iterateStep * self.iterations + sortStep
 
 
if __name__ == '__main__':
               pageRank().run()                             

Overwriting init_pr.py


##### Execute 

We run the code now for the test dataset. 

In [40]:
!python driver.py --iterations=10


	Running numNodes...
No handlers could be found for logger "mrjob.runner"
	Complete: numNodes
NUMNODES: 11

	Running pageRank...
	Complete: pageRank
ID: C 	 PR: 0.30973
ID: B 	 PR: 0.30362
ID: E 	 PR: 0.06821
ID: D 	 PR: 0.03298
ID: F 	 PR: 0.03298
ID: A 	 PR: 0.02765
ID: G 	 PR: 0.01364
ID: I 	 PR: 0.01364
ID: H 	 PR: 0.01364
ID: K 	 PR: 0.01364
ID: J 	 PR: 0.01364


### Question 9.3 

*Run your PageRank implementation on the Wikipedia dataset for 10 iterations,
and display the top 100 ranked nodes (with alpha = 0.85).*

*Run your PageRank implementation on the Wikipedia dataset for 50 iterations,
and display the top 100 ranked nodes (with teleportation factor of 0.15). 
Have the top 100 ranked pages changed? Comment on your findings. Plot the pagerank values for the top 100 pages resulting from the 50 iterations run. Then plot the pagerank values for the same 100 pages that resulted from the 10 iterations run.*

#### Solution: 

Below we run our PageRank for 5 and 10 iterations, respectively. 

##### 5 Iterations

In [ ]:
!python driver.py --emr --iterations=5


	Running numNodes...
No handlers could be found for logger "mrjob.conf"


##### 10 Iterations

In [17]:
!python driver.py --emr --iterations=10

True registered

	Running numNodes...
No handlers could be found for logger "mrjob.conf"
	Complete: numNodes

	Running pageRank...
	Complete: pageRank
ID: 3191491 	 PR: 0.0001
ID: 4624519 	 PR: 4e-05
ID: 6113490 	 PR: 0.00013
ID: 2437837 	 PR: 0.00013
ID: 13853369 	 PR: 4e-05
ID: 3591832 	 PR: 3e-05
ID: 9997298 	 PR: 5e-05
ID: 6076759 	 PR: 0.00012
ID: 7467127 	 PR: 3e-05
ID: 6416278 	 PR: 9e-05
ID: 5490435 	 PR: 5e-05
ID: 12038331 	 PR: 4e-05
ID: 1637982 	 PR: 6e-05
ID: 9924814 	 PR: 3e-05
ID: 994890 	 PR: 5e-05
ID: 13425865 	 PR: 0.00013
ID: 6237129 	 PR: 9e-05
ID: 9276255 	 PR: 9e-05
ID: 1523975 	 PR: 4e-05
ID: 9394907 	 PR: 5e-05
ID: 12785678 	 PR: 3e-05
ID: 1813634 	 PR: 4e-05
ID: 2778099 	 PR: 4e-05
ID: 3603527 	 PR: 7e-05
ID: 5051368 	 PR: 0.00016
ID: 6172167 	 PR: 5e-05
ID: 3069099 	 PR: 7e-05
ID: 15164193 	 PR: 0.0001
ID: 4568647 	 PR: 3e-05
ID: 9386580 	 PR: 7e-05
ID: 7990491 	 PR: 8e-05
ID: 11582765 	 PR: 5e-05
ID: 11148415 	 PR: 4e-05
ID: 10399499 	 PR: 4e-05
ID: 7835160 	 